# Passo 1: Configuração do Ambiente e Importação de Bibliotecas
Nesta etapa, importamos as bibliotecas necessárias para manipulação de imagens (OpenCV), operações matriciais (NumPy) e construção da rede neural (TensorFlow/Keras). Também configuramos o uso da GPU para acelerar o treinamento.

In [1]:
import tensorflow as tf
import os
import numpy as np
import cv2
from tensorflow.keras.utils import to_categorical
from glob import glob
from tensorflow.keras import layers, Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import pandas as pd
import matplotlib.pyplot as plt
import gc

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("GPU configurada para uso:", gpus)
    except RuntimeError as e:
        print(e)
else:
    print("Nenhuma GPU encontrada. O treinamento será feito na CPU.")

2025-12-09 20:51:29.900364: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-09 20:51:30.533598: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-09 20:51:32.578598: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


GPU configurada para uso: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


# Passo 2: Definição de Parâmetros e Caminhos
Definimos aqui as constantes do projeto.

**INPUT_SHAPE:** Resolução da imagem (256x256) e canais de cor (3 para RGB).

**N_CLASSES:** Quantidade de categorias a serem segmentadas (ex: doença vs Fundo).

Atenção: Certifique-se de que as pastas ```aug_data/images``` e ```aug_data/masks``` existem no mesmo diretório deste notebook!

In [2]:
# Caminhos dos diretórios (Ajuste conforme sua estrutura de pastas)
PASTA_IMAGENS = "aug_data/images"
PASTA_MASCARAS = "aug_data/masks"

# Parâmetros Globais
IMG_ALTURA = 256
IMG_LARGURA = 256
INPUT_SHAPE = (IMG_ALTURA, IMG_LARGURA, 3)
N_CLASSES = 2  # Ajuste conforme seu dataset (Ex: 0=Fundo, 1=Objeto)
BATCH_SIZE = 16
EPOCHS = 30

# **Passo 3: Carregamento e Pré-processamento dos Dados**
A função abaixo é responsável por parear as imagens com suas respectivas máscaras.
Nesta etapa, transformamos os arquivos de imagem (JPG/PNG) em matrizes numéricas que a rede neural consegue processar. A função `carregar_dados_segmentacao` realiza tarefas críticas para garantir a qualidade do treinamento:

**Imagens:** São convertidas de BGR (padrão OpenCV) para RGB e normalizadas (divididas por 255.0).

**Máscaras:** São lidas em escala de cinza e redimensionadas usando interpolação ```NEAREST``` para preservar os valores inteiros das classes.

1.  **Alinhamento (Sorting):** Utilizamos `sorted()` ao listar os arquivos. Isso é **obrigatório** para garantir que a imagem 1 (`img_01.jpg`) corresponda exatamente à máscara 1 (`mask_01.png`). Sem isso, o modelo aprenderia associações erradas.

2.  **Conversão de Cores:** O OpenCV carrega imagens no padrão BGR (Blue-Green-Red). Convertemos para **RGB**, que é o padrão esperado pela maioria dos modelos de visualização e pela U-Net.

3.  **Redimensionamento Inteligente (Resize):**
    * **Para Imagens:** Usamos a interpolação padrão (bilinear), que suaviza os pixels ao redimensionar.
    * **Para Máscaras:** Usamos **`cv2.INTER_NEAREST`** (Vizinho Mais Próximo).
    * *Por que?* As máscaras contêm classes inteiras (0, 1, 2). Se usarmos suavização, o redimensionamento criaria valores intermediários (ex: entre a classe 1 e 2, criaria 1.5), o que quebraria a classificação. O "Vizinho Mais Próximo" preserva os valores originais das classes.

4.  **Normalização:** Dividimos os valores dos pixels das imagens por **255.0**. Isso coloca os dados na escala entre 0 e 1, o que facilita a convergência matemática da rede neural.
    * Tons de preto (0) -> Cinza (0.1 a 0.9) -> Branco (1).


6.  **Ajuste de Dimensão:** Adicionamos uma dimensão extra às máscaras (`expand_dims`) para transformar o formato de `(Altura, Largura)` para `(Altura, Largura, 1 canal)`, compatível com a saída da rede.

In [3]:
def carregar_dados_segmentacao(caminho_imagens, caminho_mascaras, tamanho_img):
    imagens_paths = sorted(glob(os.path.join(caminho_imagens, "*.jpg")))
    mascaras_paths = sorted(glob(os.path.join(caminho_mascaras, "*.png")))

    lista_imagens = []
    lista_mascaras = []

    print(f"Encontradas {len(imagens_paths)} imagens e {len(mascaras_paths)} máscaras.")

    for img_path, mask_path in zip(imagens_paths, mascaras_paths):
        # Processamento da Imagem
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (tamanho_img[1], tamanho_img[0]))
    
        # Processamento da Máscara
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        mask = cv2.resize(mask, (tamanho_img[1], tamanho_img[0]), interpolation=cv2.INTER_NEAREST)
        
        lista_imagens.append(img)
        lista_mascaras.append(mask)
        
    # --- Converter as listas para arrays NumPy ---
    # Normalizar as imagens de entrada para o intervalo [0, 1]
    X = np.array(lista_imagens, dtype=np.float32) / 255.0
    y = np.array(lista_mascaras, dtype=np.uint8)
    y = np.expand_dims(y, axis=-1) # Adiciona canal extra: (256, 256, 1)

    return X, y

# Execução do carregamento
X_treino, y_treino = carregar_dados_segmentacao(PASTA_IMAGENS, PASTA_MASCARAS, (IMG_ALTURA, IMG_LARGURA))
print(f"Shape X: {X_treino.shape} | Shape y: {y_treino.shape}")

# Se houver variáveis temporárias pesadas que não vai usar mais:
if 'lista_imagens' in locals(): del lista_imagens
if 'lista_mascaras' in locals(): del lista_mascaras

# Força a limpeza
gc.collect()
print("Memória limpa após carregamento dos dados.")

Encontradas 2940 imagens e 2940 máscaras.
Shape X: (2940, 256, 256, 3) | Shape y: (2940, 256, 256, 1)
Memória limpa após carregamento dos dados.


# Passo 4: Construção da Arquitetura U-Net
Nesta etapa, definimos a estrutura da nossa Rede Neural. A U-Net possui esse nome devido ao seu formato em "U", composto por um caminho de contração (encoder) e um caminho de expansão (decoder).

O código abaixo é dividido em duas funções essenciais:
1.  **`conv_block` (Bloco Convolucional):**
    É o "tijolo" básico da construção. Cada bloco aplica duas vezes a sequência:
    * **Convolução (Conv2D):** Filtros que "escaneiam" a imagem para extrair características (bordas, texturas, formas).
    * **Batch Normalization:** Ajusta os valores intermediários para tornar o treinamento mais rápido e estável.
    * **Ativação ReLU:** Zera valores negativos, introduzindo a não-linearidade necessária para aprender padrões complexos.

2.  **`build_unet` (Montagem da Rede):**
    * **Encoder (Lado Esquerdo):** Reduz a dimensão espacial da imagem (usando `MaxPooling`) enquanto aumenta a profundidade (número de filtros). O objetivo aqui é entender **"O QUE"** está na imagem (contexto).
    * **Bottleneck (Gargalo):** A base do "U", onde a imagem está em sua representação mais compacta e abstrata.
    * **Decoder (Lado Direito):** Aumenta a imagem de volta ao tamanho original (usando `Conv2DTranspose`). O objetivo é recuperar **"ONDE"** estão os objetos (localização).
    * **Skip Connections (`concatenate`):** Este é o segredo da U-Net. Conectamos as camadas do início (ricas em detalhes visuais) diretamente às camadas do final. Isso ajuda a rede a desenhar contornos precisos na segmentação.
 
A função ```build_unet``` constrói a rede neural. Note a estrutura simétrica:

**Downsampling:** Blocos de convolução seguidos de ```MaxPooling```.

**Bottleneck:** A parte mais profunda da rede.

**Upsampling:** ```Conv2DTranspose``` seguido de concatenação (```concatenate```) com as camadas anteriores para recuperar detalhes espaciais perdidos.

In [4]:
# Limpa qualquer modelo antigo que esteja "preso" na GPU
tf.keras.backend.clear_session()
gc.collect()
print("Sessão do Keras reiniciada e GPU limpa.")
# -----------------------------
def conv_block(input_tensor, num_filters):
    x = layers.Conv2D(num_filters, (3, 3), padding="same")(input_tensor)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.Conv2D(num_filters, (3, 3), padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    return x

def build_unet(input_shape, n_classes):
    inputs = layers.Input(shape=input_shape)

    # Encoder
    s1 = conv_block(inputs, 32); p1 = layers.MaxPooling2D((2, 2))(s1)
    s2 = conv_block(p1, 64); p2 = layers.MaxPooling2D((2, 2))(s2)
    s3 = conv_block(p2, 128); p3 = layers.MaxPooling2D((2, 2))(s3)
    s4 = conv_block(p3, 256); p4 = layers.MaxPooling2D((2, 2))(s4)

    # Bottleneck
    b1 = conv_block(p4, 512)

    # Decoder
    u6 = layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding="same")(b1)
    u6 = layers.concatenate([u6, s4])
    c6 = conv_block(u6, 256)

    u7 = layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding="same")(c6)
    u7 = layers.concatenate([u7, s3])
    c7 = conv_block(u7, 128)

    u8 = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding="same")(c7)
    u8 = layers.concatenate([u8, s2])
    c8 = conv_block(u8, 64)

    u9 = layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding="same")(c8)
    u9 = layers.concatenate([u9, s1])
    c9 = conv_block(u9, 32)

    outputs = layers.Conv2D(n_classes, (1, 1), activation='softmax')(c9)

    return Model(inputs=inputs, outputs=outputs, name="U-Net")

model = build_unet(INPUT_SHAPE, N_CLASSES)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Sessão do Keras reiniciada e GPU limpa.


I0000 00:00:1765324336.652656     721 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1765 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


Model: "U-Net"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 256, 256,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 256, 256,  │        896 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 256, 256,  │        128 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 256, 256,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 256, 256,  │      9,248 │ activation[0][0]  │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 256, 256,  │        128 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 256, 256,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 128, 128,  │          0 │ activation_1[0][… │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 128, 128,  │     18,496 │ max_pooling2d[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │        256 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 128, 128,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 128, 128,  │     36,928 │ activation_2[0][… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │        256 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 128, 128,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 64, 64,    │          0 │ activation_3[0][… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 64, 64,    │     73,856 │ max_pooling2d_1[… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64, 64,    │        512 │ conv2d_4[0][0]  

 Total params: 7,771,906 (29.65 MB)

 Trainable params: 7,766,018 (29.63 MB)

 Non-trainable params: 5,888 (23.00 KB)

# Passo 5: Das funções de plotagem de resultados 
Nesta etapa final, definimos duas funções cruciais para interpretar o desempenho do modelo treinado:

1.  **`mostrar_resultados` (Análise Qualitativa):**
    Esta função realiza uma inspeção visual para validarmos se o modelo está segmentando corretamente os objetos.
    * **Seleção:** Escolhe aleatoriamente um número de amostras (`n_amostras`) do conjunto de teste.
    * **Predição:** O modelo gera as máscaras de probabilidade para essas imagens.
    * **Processamento:** Utilizamos `np.argmax` para converter as probabilidades em classes finais (ex: pixel com maior valor se torna a classe definitiva) e `np.squeeze` para ajustar as dimensões dos arrays.
    * **Visualização:** Plota lado a lado a **Imagem Original**, a **Máscara Verdadeira** (gabarito humano) e a **Predição do Modelo** (IA), permitindo identificar onde o modelo acertou ou errou.

2.  **`plot_history` (Análise Quantitativa):**
    Esta função gera gráficos de linha para diagnosticar o treinamento ao longo das épocas.
    * **Acurácia:** Deve subir tanto para treino quanto para validação.
    * **Loss (Perda):** Deve cair progressivamente.
    * **Diagnóstico:** Se a linha de treino melhorar muito e a de validação estagnar ou piorar, isso é um sinal claro de *overfitting* (o modelo decorou os dados mas não generaliza bem).

In [5]:
def mostrar_resultados(model, X, y_true, n_amostras=3, n_classes=3):
    idxs = np.random.choice(len(X), n_amostras, replace=False)
    y_pred = model.predict(X[idxs])
    y_pred_classes = np.argmax(y_pred, axis=-1)
    y_true_classes = np.squeeze(y_true[idxs], axis=-1)

    for i, idx in enumerate(idxs):
        plt.figure(figsize=(12, 4))
        # Imagem Original
        plt.subplot(1, 3, 1)
        plt.imshow(X[idx])
        plt.title("Imagem de Entrada")
        plt.axis('off')
        # Máscara Real (Ground Truth)
        plt.subplot(1, 3, 2)
        plt.imshow(y_true_classes[i], cmap='jet', vmin=0, vmax=n_classes-1)
        plt.title("Máscara Verdadeira")
        plt.axis('off')
        # Predição
        plt.subplot(1, 3, 3)
        plt.imshow(y_pred_classes[i], cmap='jet', vmin=0, vmax=n_classes-1)
        plt.title("Predição do Modelo")
        plt.axis('off')
        plt.show()

def plot_history(history):
    plt.figure(figsize=(12,5))
    # Acurácia
    plt.subplot(1,2,1)
    plt.plot(history.history['accuracy'], label='Treino')
    plt.plot(history.history['val_accuracy'], label='Validação')
    plt.title('Acurácia')
    plt.legend()
    # Loss
    plt.subplot(1,2,2)
    plt.plot(history.history['loss'], label='Treino')
    plt.plot(history.history['val_loss'], label='Validação')
    plt.title('Loss')
    plt.legend()
    plt.show()


# Passo 6: Treinamento do Modelo
Chegamos à etapa crucial onde o modelo efetivamente aprende. Antes de chamar a função de treino (`fit`), precisamos realizar três configurações importantes:

1.  **Preparação das Máscaras (One-Hot Encoding):**
    A U-Net, na camada de saída, possui `N` filtros (um para cada classe) e usa ativação *Softmax*. Portanto, ela não espera um único número inteiro por pixel (ex: 2), mas sim um vetor de probabilidades (ex: `[0, 0, 1]`).
    * Usamos `to_categorical` para transformar nossos mapas de inteiros nesse formato vetorial.
    * Aplicamos `np.clip` por segurança, garantindo que nenhum valor na máscara exceda o número de classes definido, o que causaria erro.

Antes de treinar, preparamos os targets usando One-Hot Encoding. Utilizamos **Callbacks** para otimizar o processo:

2.  **Configuração de Callbacks (Automação):**
    Callbacks são "agentes" que monitoram o treinamento em tempo real:
    * **`EarlyStopping`:** Monitora a perda na validação (`val_loss`). Se o modelo parar de melhorar por 3 épocas seguidas, ele interrompe o treino. Isso economiza tempo e evita *overfitting*.
    * **`ModelCheckpoint`:** Salva o arquivo do modelo apenas quando ele bate um recorde de melhor desempenho. Isso garante que, mesmo que o modelo piore no final do treino, você terá a melhor versão salva no disco.

3.  **Execução do Treino (`.fit`):**
    Alimentamos a rede com as imagens (`X`) e as máscaras tratadas (`y`). Separamos 10% (`validation_split=0.1`) dos dados para que os callbacks possam testar o desempenho em dados não vistos.



In [ ]:
# Tratamento das máscaras (Garantir range correto e One-Hot)
y_treino_squeezed = np.squeeze(y_treino, axis=-1)
y_treino_squeezed = np.clip(y_treino_squeezed, 0, N_CLASSES-1)
y_treino_cat = to_categorical(y_treino_squeezed, N_CLASSES)

class GarbageCollectorCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        gc.collect() # Limpa a RAM ao final de cada época

# Callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
checkpoint = ModelCheckpoint("best_unet_model.keras", monitor='val_loss', save_best_only=True, verbose=1)

# Iniciar Treinamento
history = model.fit(
    X_treino,
    y_treino_cat,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_split=0.1, # 10% dos dados para validação
    callbacks=[early_stop, checkpoint],
    verbose=1
)

# Passo 7: Avaliação e Visualização dos Resultados
Por fim, plotamos as curvas de aprendizado (Acurácia e Perda) e visualizamos as predições do modelo em imagens aleatórias do dataset.

In [6]:
# Cria um DataFrame com o histórico
history_df = pd.DataFrame(history.history)
# Salva em CSV
history_df.to_csv("unet_training_history.csv", index=False)
print("Histórico de treinamento salvo em 'unet_training_history.csv'.")

mostrar_resultados(model, X_treino, y_treino, n_amostras=3, n_classes=N_CLASSES)
plot_history(history)

NameError: name 'history' is not defined